In [48]:
!pip install pyspark

In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [50]:
spark = SparkSession.builder.config('spark.driver.host','localhost').appName('test').getOrCreate()

<font size='7'> <b>1.</b></font><font size='5'> <span style="line-height:1.7">**site02.json**을 로드한 후, pokemon의 <font color='blue'>**name**, **Height**, **Weight**, **type**, **weaknesses**</font>로 구성된 <b><font color='red'>DataFrame</font></b>를 찾으시오.</span>

In [51]:
df = spark.read.json("site02.json")

In [52]:
df = df.select(
    "name",
    col("info.Height").alias("Height"),
    col("info.Weight").alias("Weight"),
    "type",
    "weaknesses",
)

In [53]:
df.show()

+----------+------+--------+--------------------+--------------------+
|      name|Height|  Weight|                type|          weaknesses|
+----------+------+--------+--------------------+--------------------+
|      NULL|  NULL|    NULL|                NULL|                NULL|
| Bulbasaur| 0.7 m|  6.9 kg|     [Grass, Poison]|[Fire, Psychic, F...|
|   Ivysaur| 1.0 m| 13.0 kg|     [Grass, Poison]|[Fire, Psychic, F...|
|  Venusaur| 2.0 m|100.0 kg|[Grass, Poison, G...|[Fire, Psychic, F...|
|Charmander| 0.6 m|  8.5 kg|              [Fire]|[Water, Ground, R...|
|Charmeleon| 1.1 m| 19.0 kg|              [Fire]|[Water, Ground, R...|
| Charizard| 1.7 m| 90.5 kg|[Fire, Flying, Fi...|[Water, Electric,...|
|  Squirtle| 0.5 m|  9.0 kg|             [Water]|   [Grass, Electric]|
| Wartortle| 1.0 m| 22.5 kg|             [Water]|   [Grass, Electric]|
| Blastoise| 1.6 m| 85.5 kg|[Water, Water, Wa...|[Grass, Electric,...|
|  Caterpie| 0.3 m|  2.9 kg|               [Bug]|[Fire, Flying, Rock]|
|   Me

<font size='7'> <b>2.</b></font><font size='5'> <span style="line-height:1.7">1번에서 찾은 DataFrame에서 <font color='blue'><b>name</b>이나 <b>type</b> column이 **null**인 행</font>은 <font color='red'>**삭제**</font>하고, **삭제된 행의 수**를 찾으시오.

In [54]:
df_filtered = df.dropna(subset=["name", "type"])

In [55]:
deleted_rows = df.count() - df_filtered.count()

In [56]:
print(f"Deleted rows: {deleted_rows}")

Deleted rows: 2


<font size='7'> <b>3.</b></font><font size='5'> <span style="line-height:1.7">2번에서 찾은 DataFrame에 <font color='blue'><b>BMI</b> column을 추가</font>하고, **BMI 기준 내림차순으로 정렬**된 DataFrame을 찾으시오.</span>
> - BMI를 계산하기 위해, **Weight**와 **Height**의 **단위(m, kg)를 삭제**하고 **float type으로 변환**해야 함
> - **BMI** = **Weight** / **(Height * Height)**

In [57]:
df_filtered = df_filtered.withColumn("Weight", regexp_replace(col("Weight"), " kg", "").cast("float"))
df_filtered = df_filtered.withColumn("Height", regexp_replace(col("Height"), " m", "").cast("float"))

In [58]:
df_filtered = df_filtered.withColumn("BMI", col("Weight") / (col("Height") * col("Height")))

In [59]:
df_filtered = df_filtered.sort(col("BMI").desc())

In [60]:
df_filtered.show()

+----------+------+------+--------------------+--------------------+------------------+
|      name|Height|Weight|                type|          weaknesses|               BMI|
+----------+------+------+--------------------+--------------------+------------------+
|   Cosmoem|   0.1| 999.9|           [Psychic]|  [Ghost, Dark, Bug]| 99989.99536406282|
|    Minior|   0.3|  40.0|[Rock, Flying, Ro...|[Steel, Water, El...|444.44442678380966|
|      Aron|   0.4|  60.0|       [Steel, Rock]|[Water, Fighting,...| 374.9999734573085|
|    Durant|   0.3|  33.0|        [Bug, Steel]|            [Fire, ]|366.66665209664296|
|  Clamperl|   0.4|  52.5|             [Water]|   [Grass, Electric]|328.12497677514494|
|   Torkoal|   0.5|  80.4|              [Fire]|[Water, Ground, R...| 321.6000061035156|
|    Cacnea|   0.4|  51.3|             [Grass]|[Fire, Flying, Ic...| 320.6249725376275|
|  Munchlax|   0.6| 105.0|            [Normal]|          [Fighting]| 291.6666550768751|
| Sandygast|   0.5|  70.0|     [

<font size='7'> <b>4.</b></font><font size='5'> <span style="line-height:1.7">3번에서 찾은 DataFrame의 <font color='blue'><b>BMI</b>, <b>height</b>, <b>weight</b></font> column <font color='red'><b>평균값</b></font>을 찾으시오.

In [62]:
df_filtered.select(mean("BMI"), mean("Height"), mean("Weight")).show()


+------------------+------------------+------------------+
|          avg(BMI)|       avg(Height)|       avg(Weight)|
+------------------+------------------+------------------+
|152.09968383871913|1.1829787265480165|62.987570057565726|
+------------------+------------------+------------------+

